In [5]:
import os
import cv2
import numpy as np
from keras.models import load_model
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import imageio
import keras.utils as image

In [13]:
def pre_process_img(imagepath):
    
        up_img = Image.open(imagepath)
        up_img = up_img.resize((512, 512), Image.ANTIALIAS)
        up_array = image.img_to_array(up_img)
        up_array = np.expand_dims(up_array, axis=0)
        up_array /= 255.

        return up_array

def img_enh(imagepath, modelpath):
    processedImage = pre_process_img(imagepath)
    og_img = Image.open(imagepath)
    orig_image = np.asarray(og_img)
    enh_model = load_model(modelpath, compile=False)

    # Enhancing the image
    gen_image = enh_model.predict(processedImage)
    gen_image = gen_image.reshape(gen_image.shape[1:])

    final_image = cv2.resize(gen_image,
                             (int(orig_image.shape[1]/2), int(orig_image.shape[0]/2)),
                             interpolation=cv2.INTER_AREA)
    final_image = cv2.normalize(final_image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    temp_path = os.path.join('D:/FYP/Phaedra_BE', 'editedimage_1.jpg')
    imageio.imwrite(temp_path, final_image)
    enhanced_image = cv2.imread(temp_path)

    # Resize the original image to match the enhanced image
    resized_orig_image = cv2.resize(orig_image, (enhanced_image.shape[1], enhanced_image.shape[0]))

    return resized_orig_image, enhanced_image

def mse(x, y):
    return np.mean((x - y) ** 2)

def psnr(x, y, max_pixel_value=255.0):
    mse_value = mse(x, y)
    return 20 * np.log10(max_pixel_value) - 10 * np.log10(mse_value)

In [14]:
imagepath = 'D:/FYP/Phaedra_BE/65d.jpg'
modelpath = 'EnhancementModel2_up.h5'

resized_orig_image, enhanced_image = img_enh(imagepath, modelpath)

# Calculate the metrics
psnr_value = psnr(resized_orig_image, enhanced_image)
ssim_value = ssim(resized_orig_image, enhanced_image, multichannel=True)
mse_value = mse(resized_orig_image, enhanced_image)

print("PSNR:", psnr_value)
print()
print("SSIM:", ssim_value)
print()
print("MSE:", mse_value)

C:\Users\Ph03nix6\AppData\Local\Temp\ipykernel_31280\202331273.py:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  up_img = up_img.resize((512, 512), Image.ANTIALIAS)


1/1 [==============================] - 1s 1s/step


C:\Users\Ph03nix6\AppData\Local\Temp\ipykernel_31280\1355721650.py:8: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(resized_orig_image, enhanced_image, multichannel=True)


PSNR: 29.056469723901856

SSIM: 0.5549690583495912

MSE: 80.80409833333333
